In [ ]:
!pip install pyngrok

In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 24.9 MB/s eta 0:00:00


In [ ]:
!pip install plotly

In [ ]:
from pyngrok import ngrok
ngrok.kill()
NGROK_AUTH_TOKEN = "2jzHK8uFaivgCMbEqved3SXcLUz_4RWTDdMvETjeW73jr43Kz"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
ngrok.connect(8050)

<NgrokTunnel: "https://a810-35-230-49-53.ngrok-free.app" -> "http://localhost:8050">

In [ ]:
#!pip install dash
#!pip install plotly
import pandas as pd
import dash
from dash import html as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import contextlib
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a dash application
app = dash.Dash(__name__)

#REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

#Supress the commnad outputs
with contextlib.redirect_stdout('Worked'):
	pass

# Read the student data into pandas dataframe
df = pd.read_csv('/content/Student_performance_data.csv')
df_actual = df.copy() #keepinf a copy to use the original below

#Updating the Categorical Values in the data frame
df['Gender']            = df['Gender'].replace({0:'Male', 1:"Female"}).astype(str)
df['Ethnicity']         = df['Ethnicity'].replace({0:'Caucasian', 1:"African American", 2:"Asian", 3:"Other"}).astype(str)
df['ParentalSupport']   = df['ParentalSupport'].replace({0: "None", 1: "Low", 2: "Moderate", 3: "High", 4: "Very High"}).astype(str)
df['Tutoring']          = df['Tutoring'].replace({0:'No', 1:'Yes'}).astype(str)
df['Extracurricular']   = df['Extracurricular'].replace({0:'No',1:'Yes'}).astype(str)
df['Sports']            = df['Sports'].replace({0:'No', 1:'Yes'}).astype(str)
df['Music']             = df['Music'].replace({0:'No', 1:'Yes'}).astype(str)
df['Volunteering']      = df['Volunteering'].replace({0:'No', 1:'Yes'}).astype(str)
df['ParentalEducation'] = df['ParentalEducation'].replace({0: "None", 1: "High School", 2: "Some College", 3: "Bachelor", 4: "Higher"}).astype(str)
df['GradeClass']        = df['GradeClass'].replace({0: "A", 1: "B", 2: "C", 3: "D", 4: "F"}).astype(str)

#Create the list of reports
report_list = ['On Categorical Factors', 'On Quantitative Factors', 'Other Graphs']

colors = {
    'background': '#000000',
    'text': '#7FDBFF'
}

#Application layout
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
	#Add title to the dashboard
    html.H1(children='Student Performance', style={'textAlign':'center', 'background':'#000000', 'color':colors['text']}),

    #Add DropDown
	html.Div(children='Report Options', style={'textAlign':'left', 'font-size':'20px', 'color':colors['text']}),
	dcc.Dropdown(
		id='dropdown-reports',
		options=[{'label': i, 'value': i} for i in report_list],
		placeholder='Select a report',
		style={'width': '100%', 'padding':'3px','font-size':'20px','text-align-last':'center',
			   'backgroundColor':'#a2ddf5','color':colors['text']},
		),

   	#Container for output
 	html.Div(
    	children=[
        	html.Div(
				children=[
					html.Div([], id='plot1', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot2', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot3', style={'width': '400px','height': '400px'}),
				], style={'display': 'flex', 'flexDirection': 'row', 'height' : '400px'}
        	),
       		html.Div(
				children=[
					html.Div([], id='plot4', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot5', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot6', style={'width': '400px','height': '400px'}),
 				],  style={'display': 'flex', 'flexDirection': 'row', 'height' : '400px'}
        	),
            html.Div(
				children=[
					html.Div([], id='plot7', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot8', style={'width': '400px','height': '400px'}),
					html.Div([], id='plot9', style={'width': '400px','height': '400px'}),
 				],  style={'display': 'flex', 'flexDirection': 'row', 'height' : '400px'}
        	),
    	], style={'display': 'flex', 'flexDirection': 'column'}
	),
]),

# Callback function definition
@app.callback([
     			Output('plot1', 'children'),
               	Output('plot2', 'children'),
               	Output('plot3', 'children'),
               	Output('plot4', 'children'),
               	Output('plot5', 'children'),
                Output('plot6', 'children'),
                Output('plot7', 'children'),
               	Output('plot8', 'children'),
                Output('plot9', 'children')
             ],
              [
                Input('dropdown-reports', 'value')
              ]
             )
#Callback function and return graph
def get_graph(chosen_report):
    if chosen_report == 'On Categorical Factors':
          #Plot1
          ddf = df[['Ethnicity', 'Gender']].value_counts().to_frame().reset_index()
          ddf.columns = ['Ethnicity', 'Gender', 'Count']
          new_df = ddf.pivot(index='Ethnicity', columns='Gender')['Count'].fillna(0)
          fig1 = px.imshow(new_df,
            	labels=dict(x="Gender", y="Ethnicity", color="count"),
            	x=['Male', 'Female'],
            	y=['Caucacian', 'African Americacn', 'Asian', 'Other']
        )
          fig1.update_scenes(					# Update 3D scene options
				aspectratio=dict(x=1, y=1, z=0.7),
				aspectmode="manual"
		)
          fig1.update_xaxes(side="bottom")
          fig1.update_layout(
				width = 400,
				height = 400,
				autosize=False,
				title = 'Gender Distribution',
				title_x = 0.5,
				xaxis_title = 'Gender',
				yaxis_title = 'Ethnicity',
				template="plotly_dark",
				coloraxis_colorbar_title="Students Count",
				updatemenus=[
				dict(
					buttons=list([
					dict(
					args=["type", "surface"],
					label="3D Surface",
					method="restyle"
				),
				dict(
					args=["type", "heatmap"],
					label="Heatmap",
					method="restyle"
				)
			]),
				direction="down",
				pad={"r": 10, "t": 10},
				showactive=True,
				x=0.1,
				xanchor="left",
				y=1.1,
				yanchor="top"
			),
			]
				)
          #Plot2
          fig2 = px.box(df, x='Tutoring', y="GPA", color="Tutoring")
          fig2.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="Tutoring Vs GPA",
            title_x=0.5,
            template='plotly_dark',
            updatemenus=[
        			dict(
						type="buttons",
						direction="left",
						buttons=list([
						dict(
							args=["type", "box"],
							label="Box Plot",
							method="restyle"
						),
						dict(
							args=["type", "bar"],
							label="Bar Chart",
							method="restyle"
                		)
            		]),
        		)]
    		)
          #Plot3
          fig3= px.box(df, x="Gender", y="GPA", points="all", notched=True, color="Gender")
          fig3.update_layout(
    		title="Gender Vs GPA with datapoints",
            title_x=0.5,
            template='plotly_dark'
		)
          #Plot4
          df1 = df[['GradeClass', 'GPA']]
          df1 = df1.sort_values(by=['GradeClass', 'GPA'], ignore_index=True)
          fig4 = px.box(df1, x="GradeClass", y="GPA", points = False, color="GradeClass")
          fig4.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="GradeClass Vs GPA",
            title_x=0.5,
            template='plotly_dark',
		)
          #Plot5
          fig5 = px.box(df, x='Ethnicity', y="GPA", color="Ethnicity")
          fig5.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="Ethnicity Vs GPA",
            title_x=0.5,
            template='plotly_dark'
		)
          #Plot6
          fig6 = px.box(df, x='ParentalSupport', y="GPA", color="Gender")
          fig6.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="ParentalSupport Vs GPA",
            title_x=0.5,
            template='plotly_dark'
		)
          #Plot7
          fig7 = px.box(df, x='ParentalEducation', y="GPA")
          fig7.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="ParentalEducation Vs GPA",
            title_x=0.5,
            template='plotly_dark'
		)
          #Plot8
          fig8 = go.Figure()
          fig8.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['ParentalEducation'],
			name='ParentalEducation',
			marker_color='#3D9970'
		))
          fig8.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['ParentalSupport'],
			name='ParentalSupport',
			marker_color='#FF4136'
		))
          fig8.update_layout(
			autosize=False,
			width=400,
			height=400,
            boxmode='group',
			title='Comparison'+'<br>'+'ParentalEducation/Support Vs GPA',
			title_x=0.5,
			yaxis_title='GPA',
			xaxis_title='ParentalEducation/Support',
			showlegend=True,
            template='plotly_dark'
		)
          #Plot9
          fig9 = go.Figure()
          fig9.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['Sports'],
			name='Sports',
			marker_color='#3D9970'
		))
          fig9.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['Music'],
			name='Music',
			marker_color='#FF4136'
		))
          fig9.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['Extracurricular'],
			name='Extracurricular',
			marker_color='#d4d45b'
		))
          fig9.add_trace(go.Box(
			y=df_actual['GPA'],
			x=df_actual['Volunteering'],
			name='Voluteering',
			marker_color='#5b63d4'
		))
          fig9.update_layout(
			autosize=False,
			width=400,
			height=400,
            boxmode='group',
			title='Extracurricular/Sports/' + '<br>' + 'Music/Volunteering Vs GPA',
			title_x=0.5,
			yaxis_title='GPA',
			xaxis_title='Extracurricular/Sports/' + '<br>' + 'Music/Volunteering Vs GPA',
			showlegend=True,
            template='plotly_dark'
		)
          return [
            dcc.Graph(figure=fig1),
            dcc.Graph(figure=fig2),
            dcc.Graph(figure=fig3),
            dcc.Graph(figure=fig4),
            dcc.Graph(figure=fig5),
            dcc.Graph(figure=fig6),
            dcc.Graph(figure=fig7),
            dcc.Graph(figure=fig8),
            dcc.Graph(figure=fig9)
        ]
    elif chosen_report == 'On Quantitative Factors':	#Numerical factors
          #Plot1
          df2 = df[['StudyTimeWeekly', 'GPA']]
          df2['GPA'] = df2['GPA'].round(0).astype(int)
          box_trace = go.Box(
				y=df2.StudyTimeWeekly,
				x=df2.GPA,
				name='StudyTimeWeekly',
        )
          fig1 = go.Figure(box_trace)
          fig1.update_layout(
    		title='Study Time Weekly Hours Vs GPA',
    		xaxis_title=dict(text='GPA'),
    		yaxis_title=dict(text='Study Hours'),
            title_x=0.5,
            template='plotly_dark'
 		)

          #Plot2
          fig2 = px.box(df, x='Absences', y='GPA')
          fig2.update_layout(
            autosize=True,
            width=600,
			height=400,
            title="Absences Vs GPA",
            title_x=0.5,
            xaxis=dict(rangeslider=dict(visible=True), type="linear"),
            template='plotly_dark'
		)

		  #Plot3 Empty Figure
          fig0 = go.Figure()

		  #Plot4
          x0 = df['StudyTimeWeekly']
          x1 = df['Absences']
          fig4 = go.Figure()
          fig4.add_trace(go.Box(x=x0, name='Study Time<br>Weekly Hours'))
          fig4.add_trace(go.Box(x=x1, name='Absences Days'))
          fig4.update_layout(
			height=400,
			width=800,
			title_text='Subplots Study Hours/Absences',
			title_x=0.5,
			showlegend=True,
            template='plotly_dark'
   		)

		#   #Plot5 Empty Figure

		#   #Plot6
          fig6 = px.box(df, y="GPA")
          fig6.update_layout(
            autosize=False,
			width=400,
			height=400,
            title="GPA Distribution",
            title_x=0.5,
            template='plotly_dark'
		)

		  #Plot7
          fig7 = px.scatter(df, x='StudyTimeWeekly', y='GPA', opacity=0.65,
    						trendline='ols', trendline_color_override='red'
			)
          fig7.update_layout(
			height=400,
			width=800,
			title='StudyTimeWeekly Vs GPA with Regression Line',
			title_x=0.5,
			showlegend=False,
            template='plotly_dark'
		)

          return [
			dcc.Graph(figure=fig1),
			dcc.Graph(figure=fig2),
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig4),
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}), #Empty figure
			dcc.Graph(figure=fig6),
			dcc.Graph(figure=fig7),
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
        ]
    elif chosen_report == 'Other Graphs':	#Other Charts
          #Plot1
          fig1 = px.pie(df,
                        values='GPA',
                        names='Ethnicity',
             			title='Ethnicity Vs GPA',
             			hover_data=['GPA'],
                        labels={'GPA':'GPA'})
          fig1.update_traces(textposition='inside',
                             textinfo='percent+label')
          fig1.update_layout(
                autosize=False,
				width=400,
				height=400,
                template='plotly_dark'
			)

          #Plot2
          fig2 = px.histogram(df.GPA, x="GPA", nbins=30, title="Distribution of GPA")
          fig2.update_layout(
                autosize=False,
				width=700,
				height=700,
                template='plotly_dark'
			)

		  #Empty Figure
          fig0 = go.Figure()

          return [
			dcc.Graph(figure=fig1),
 			dcc.Graph(figure=fig2),
            dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}), #Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
			dcc.Graph(figure=fig0, style={'visibility': 'hidden'}),	#Empty figure
        ]
    else:
        return [html.Div()] * 12

# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>